### No Feature Selection!

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_selection import RFECV
import math,os

In [2]:
#Load the train and test data
files = os.listdir("Data")
filename = files[-1]
train_data = pd.read_csv("./Data/{}".format(filename),header=None)
file_list = files[:-1] # test data list
train_output = train_data[len(train_data.columns)-1]
del train_data[len(train_data.columns)-1]

In [3]:
# Initialize the models
reg = LinearRegression()
rf = RandomForestRegressor()
regressors = [reg,rf]
regressor_names = ["Linear Regression","Random Forests"]

In [4]:
for regressor,regressor_name in zip(regressors,regressor_names):
    res = []
    col_names = ["fileName", "Mean Squared Error", "R2 score", "HIT@10"]
    for filename in file_list:
        # Load the test data
        test_data = pd.read_csv("./Data/{}".format(filename),header=None)
        test_output = test_data[len(test_data.columns)-1]
        del test_data[len(test_data.columns)-1]
        rank_test = [index for index,value in sorted(list(enumerate(test_output)),key=lambda x:x[1], reverse=True)]
        regressor.fit(train_data,train_output)
        predicted_values = regressor.predict(test_data)
        rank_predict = [index for index,value in sorted(list(enumerate(predicted_values)),key=lambda x:x[1], reverse=True)]
        counter = len([x for x in rank_predict[:10] if x in rank_test[:10]])

        result = [filename, metrics.mean_squared_error(test_output,predicted_values), metrics.r2_score(test_output,predicted_values), counter]
        res.append(result)
    df = pd.DataFrame(data=res, columns=col_names)
    col_mean = df[["Mean Squared Error", "R2 score", "HIT@10"]].mean()
    col_mean["fileName"] = "Average"
    df = df.append(col_mean, ignore_index=True)
    df.to_csv("./{}123.csv".format(regressor_name), index=False)
    print ("Mean Squared Error for ",regressor_name, " : ", col_mean["Mean Squared Error"])
    print ("R2 score for ",regressor_name, " : ", col_mean["R2 score"])
    print ("HIT@10 for ",regressor_name, " : ", col_mean["HIT@10"])
    print("\n")

NameError: name 'selected' is not defined